# 📊 BA-02: Planeación S&OP con escenarios de demanda

Modelo simplificado de S&OP que integra demanda pronosticada, capacidad y políticas de servicio para simular escenarios (base/optimista/pesimista) y evaluar backlog, utilización y nivel de servicio.

---\nid: "BA-02"\ntitle: "Planeación S&OP con escenarios de demanda"\nspecialty: "Business Analytics"\nprocess: "Plan"\nlevel: "Intermediate"\ntags: ["sop", "scenarios", "capacity", "backlog", "fill-rate"]\nestimated_time_min: 45\n---\n

In [ ]:
# ⚙️ Resolución robusta de rutas
from pathlib import Path
def resolve_repo_root(start: Path) -> Path:
    cur = start
    for _ in range(6):
        if (cur / 'data').exists() and (cur / 'notebooks').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start

root = resolve_repo_root(Path.cwd())
raw_path = root / 'data' / 'raw'
processed_path = root / 'data' / 'processed'
processed_path.mkdir(parents=True, exist_ok=True)

### ⚙️ Preparación de entorno y rutas
Este bloque detecta la raíz del repositorio y define las rutas `data/raw` y `data/processed` para garantizar que el notebook sea ejecutable en VS Code y con papermill.

In [ ]:
# 📦 Generación de datos sintéticos si no existen
import pandas as pd
import numpy as np
np.random.seed(42)

weeks = pd.date_range('2025-01-06', periods=12, freq='W-MON')
families = ['A', 'B', 'C']
skus = [f'{fam}{i:02d}' for fam in families for i in range(1, 6)]

# Demanda semanal por SKU (base)
dem_base = []
for sku in skus:
    fam = sku[0]
    mean = {'A': 120, 'B': 80, 'C': 60}[fam]
    std = {'A': 20, 'B': 15, 'C': 10}[fam]
    vals = np.clip(np.random.normal(mean, std, len(weeks)).round(), 0, None)
    for w, v in zip(weeks, vals):
        dem_base.append({'week': w, 'family': fam, 'sku': sku, 'demand': int(v)})
df_dem_base = pd.DataFrame(dem_base)

# Capacidad por período (planta + transporte)
df_capacity = pd.DataFrame({
  'week': weeks,
  'prod_capacity': [900, 900, 950, 950, 1000, 1000, 1000, 1000, 950, 950, 900, 900],
  'trans_capacity': [800, 800, 850, 850, 900, 900, 900, 900, 850, 850, 800, 800]
})
df_capacity['total_capacity'] = df_capacity['prod_capacity'] + df_capacity['trans_capacity']

# Fill rate objetivo por familia
df_policies = pd.DataFrame({
  'family': families,
  'fill_rate_target': [0.95, 0.90, 0.88]
})

df_dem_base.to_parquet(processed_path / 'sop_dem_base.parquet', index=False)
df_capacity.to_parquet(processed_path / 'sop_capacity.parquet', index=False)
df_policies.to_parquet(processed_path / 'sop_policies.parquet', index=False)
df_dem_base.head(), df_capacity.head(), df_policies

### 📦 Generación de datos sintéticos
Se construyen demanda semanal por SKU/familia (con ruido), capacidad total (producción + transporte) y políticas de servicio (fill rate objetivo). Se persisten en `data/processed` para reutilización.

- Semilla fija: Se establece `np.random.seed(42)` para asegurar reproducibilidad.
- Demanda: Generada por familia con medias y desvíos distintos, luego recortada a valores no negativos.
- Capacidad: `total_capacity = prod_capacity + trans_capacity` por semana.
- Políticas: `fill_rate_target` por familia para comparar desempeño.

## 🎛️ Definición de escenarios
- Base: demanda observada
- Optimista: +10% demanda
- Pesimista: -10% demanda

## 📋 Qué - Por qué - Para qué - Cuándo - Cómo

### ❓ Qué
Simular S&OP semanal por familia con capacidad total y políticas de servicio.

### 🔍 Por qué
Visibilizar impacto de escenarios de demanda en backlog, utilización y nivel de servicio.

### 🎯 Para qué
Apoyar decisiones de nivel de producción/transporte y compromisos de servicio.

### ⏰ Cuándo
Ciclos S&OP semanales/mensuales, ajuste de capacidad o promociones.

### 🛠️ Cómo
Datos sintéticos, agregación por semana/familia, asignación por prioridades y cálculo de KPIs.

## 📚 Resumen y Próximos pasos
1. Modelo S&OP ejecutable con escenarios base/optimista/pesimista.
2. KPIs de backlog, utilización y nivel de servicio por familia.
3. Exportes en `data/processed/sop_scenarios/`.

Próximos pasos: agregar visualizaciones (líneas para backlog y barras para fill rate) y sensibilidad de capacidad.

In [ ]:
# Construcción de escenarios
def build_scenario(df_dem, name, factor):
    s = df_dem.copy()
    s['scenario'] = name
    s['demand'] = (s['demand'] * factor).round().astype(int)
    return s

sc_base = build_scenario(df_dem_base, 'base', 1.00)
sc_opt = build_scenario(df_dem_base, 'optimista', 1.10)
sc_pes = build_scenario(df_dem_base, 'pesimista', 0.90)
df_dem_scen = pd.concat([sc_base, sc_opt, sc_pes], ignore_index=True)
df_dem_scen.head()

### 🎛️ Construcción de escenarios de demanda
Se generan tres escenarios: base (1.00), optimista (+10%) y pesimista (-10%) para analizar sensibilidad en backlog, utilización y nivel de servicio.

In [ ]:
# Agregación y simulación semana a semana
import pandas as pd

dem_week_family = df_dem_scen.groupby(['scenario', 'week', 'family'], as_index=False)['demand'].sum()
priority = ['A', 'B', 'C']
records = []

for scen in dem_week_family['scenario'].unique():
    backlog = {fam: 0 for fam in priority}
    for row in df_capacity.itertuples(index=False):
        week = row.week
        cap = row.total_capacity
        week_dem = (
            dem_week_family[
                (dem_week_family['scenario'] == scen) & (dem_week_family['week'] == week)
            ].set_index('family')['demand'].to_dict()
        )
        need = {fam: week_dem.get(fam, 0) + backlog[fam] for fam in priority}
        delivered = {fam: 0 for fam in priority}
        remaining = cap
        # asignación por prioridad A > B > C
        for fam in priority:
            take = min(need[fam], remaining)
            delivered[fam] = take
            remaining -= take
        # actualizar backlog
        for fam in priority:
            backlog[fam] = max(need[fam] - delivered[fam], 0)
        utilization = (cap - remaining) / cap if cap > 0 else 0.0
        # generar registros por familia
        for fam in priority:
            dem = week_dem.get(fam, 0)
            fill_rate = (delivered[fam] / dem) if dem > 0 else 1.0
            records.append({
                'scenario': scen,
                'week': week,
                'family': fam,
                'demand': int(dem),
                'delivered': int(delivered[fam]),
                'backlog': int(backlog[fam]),
                'utilization': utilization,
                'fill_rate': fill_rate
            })

# resultados consolidados
df_results = pd.DataFrame(records)
df_results.head()

### 🧮 Simulación y asignación de capacidad
Se agrega demanda por semana/familia y se asigna capacidad por prioridades (A>B>C), arrastrando backlog y calculando utilización y fill rate por familia.

- Prioridad: La capacidad se asigna en orden A, luego B y C.
- Backlog: Se acumula y se arrastra semana a semana por familia.
- Utilización: `(capacidad asignada / capacidad disponible)` por semana.
- Fill rate: `delivered / demand` por familia y semana; si `demand=0`, se considera 1.0 por convención.

In [ ]:
# KPIs por escenario y familia
kpi = df_results.groupby(['scenario', 'family']).agg({
  'backlog': 'sum',
  'utilization': 'mean',
  'fill_rate': 'mean',
  'demand': 'sum',
  'delivered': 'sum'
}).reset_index()
kpi = kpi.merge(df_policies, on='family', how='left')
kpi['fill_rate_gap'] = (kpi['fill_rate'] - kpi['fill_rate_target']).round(3)
kpi.round(3)

### 📈 KPIs y comparación con objetivos
Se agregan KPIs por escenario y familia (backlog total, utilización promedio, fill rate promedio) y se compara contra objetivos definidos en `df_policies`.

- Backlog: Suma total por escenario y familia.
- Utilización: Promedio semanal por escenario.
- Fill rate: Promedio semanal por escenario y familia.
- Gap vs objetivo: `fill_rate_gap = fill_rate - fill_rate_target`.
  - Positivo: sobrecumplimiento del objetivo.
  - Negativo: incumplimiento del objetivo.

In [ ]:
# Exportar resultados
out_dir = processed_path / 'sop_scenarios'
out_dir.mkdir(parents=True, exist_ok=True)
df_results.to_parquet(out_dir / 'results.parquet', index=False)
kpi.to_parquet(out_dir / 'kpi.parquet', index=False)
print('✅ Resultados exportados a', out_dir)
kpi.head()

### 💾 Exportes de resultados
Se exportan `results.parquet` y `kpi.parquet` a `data/processed/sop_scenarios/` para análisis y visualizaciones posteriores.

- Rutas: `data/processed/sop_scenarios/results.parquet` y `data/processed/sop_scenarios/kpi.parquet`.
- Formato: Parquet para eficiencia y compatibilidad con herramientas de análisis.
- Reutilización: Facilita cargar resultados sin re-ejecutar la simulación.

In [ ]:
# 📈 Visualización: Backlog semanal por escenario
import matplotlib.pyplot as plt
backlog_week = df_results.groupby(['scenario', 'week'])['backlog'].sum().reset_index()
fig, ax = plt.subplots(figsize=(10,5))
for scen, g in backlog_week.groupby('scenario'):
    ax.plot(g['week'], g['backlog'], label=scen)
ax.set_title('Backlog semanal por escenario')
ax.set_xlabel('Semana'); ax.set_ylabel('Backlog')
ax.legend(); plt.tight_layout(); plt.show()

### 📈 Visualización de backlog semanal
Se grafica la evolución del backlog total por escenario para observar saturación y alivios en el tiempo.

- Eje Y: Unidades de backlog agregadas por semana.
- Comparabilidad: Las curvas permiten comparar carga acumulada entre escenarios.
- Uso: Detectar semanas críticas para ajustes de capacidad.

In [ ]:
# 📊 Visualización: Fill rate promedio por escenario y familia
import matplotlib.pyplot as plt
pivot_fr = kpi.pivot(index='family', columns='scenario', values='fill_rate')
fig, ax = plt.subplots(figsize=(8,5))
pivot_fr.plot(kind='bar', ax=ax)
ax.set_title('Fill rate promedio por escenario y familia')
ax.set_xlabel('Familia'); ax.set_ylabel('Fill rate')
ax.legend(title='Escenario'); plt.tight_layout(); plt.show()

### 📊 Visualización de fill rate
Se comparan los niveles de servicio promedio por familia entre escenarios para evaluar cumplimiento contra objetivos.

- Rango: Valores en [0, 1].
- Objetivo: Referencia en `df_policies.fill_rate_target` por familia.
- Uso: Identificar escenarios/familias con brecha negativa y priorizar acciones.

In [ ]:
# ✅ Sanidad básica
assert not df_results.empty, 'Resultados vacíos'
assert not kpi.empty, 'KPIs vacíos'
print('Notebook BA-02 listo: resultados y KPIs generados.')